In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [38]:
df = pd.read_csv('StudentsPerformance.csv')
df

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [39]:
df.columns = df.columns.str.replace(" ", "_").copy()
df = df.rename({'parental_level_of_education':'parent_education', 
                        'race/ethnicity':'ethnicity', 
                        'test_preparation_course':'prep_course'}, 
                        axis=1).copy()
df.columns

Index(['gender', 'ethnicity', 'parent_education', 'lunch', 'prep_course',
       'math_score', 'reading_score', 'writing_score'],
      dtype='object')

In [40]:
# Menambahkan kolom rata-rata nilai
df['average_score'] = df[['math_score', 'reading_score', 'writing_score']].mean(axis=1)

# Menampilkan data dengan kolom rata-rata nilai
df

,gender,ethnicity,parent_education,lunch,prep_course,math_score,reading_score,writing_score,average_score
0,female,group B,bachelor's degree,standard,none,72,72,74,72.666667
1,female,group C,some college,standard,completed,69,90,88,82.333333
2,female,group B,master's degree,standard,none,90,95,93,92.666667
3,male,group A,associate's degree,free/reduced,none,47,57,44,49.333333
4,male,group C,some college,standard,none,76,78,75,76.333333
...,...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95,94.000000
996,male,group C,high school,free/reduced,none,62,55,55,57.333333
997,female,group C,high school,free/reduced,completed,59,71,65,65.000000
998,female,group D,some college,standard,completed,68,78,77,74.333333


In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report


# Define independent variables (factors) and target variable (performance_level)
X = df[['gender', 'parent_education', 'lunch', 'prep_course']]
y = df['average_score']

# Convert target variable to categorical
y = pd.cut(y, bins=[0, 40, 60, 100], labels=['Low', 'Medium', 'High'])

# Convert categorical variables to dummy variables (one-hot encoding)
X = pd.get_dummies(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build KNeighborsClassifier model
model = KNeighborsClassifier()
model.fit(X_train, y_train)

# Predict performance levels for the test data
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Display classification report
classification_report = classification_report(y_test, y_pred)
print(classification_report)

# Display accuracy
print('Accuracy:', accuracy)

              precision    recall  f1-score   support

        High       0.69      0.86      0.76       136
         Low       0.00      0.00      0.00        10
      Medium       0.27      0.13      0.18        54

    accuracy                           0.62       200
   macro avg       0.32      0.33      0.31       200
weighted avg       0.54      0.62      0.57       200

Accuracy: 0.62


In [61]:
df['parent_education'].unique()
df['lunch'].unique()

array(['standard', 'free/reduced'], dtype=object)

In [63]:
# Data baru untuk pengujian prediksi
new_data = pd.DataFrame({
    'gender': ['male'],
    'parent_education': ['master\'s degree'],
    'lunch': ['free/reduced'],
    'prep_course': ['completed']
})

# Preprocess new data
new_data = pd.get_dummies(new_data)

# Check missing columns in new data
missing_columns = set(X_train.columns) - set(new_data.columns)

# Add missing columns to new data with value 0
for col in missing_columns:
    new_data[col] = 0

# Reorder columns to match the order in training data
new_data = new_data[X_train.columns]

# Make predictions on new data
predictions = model.predict(new_data)

# Print the predicted performance level
print('Predicted Performance Level:', predictions)


Predicted Performance Level: ['High']


In [65]:
import pickle

In [68]:
file_name = 'student_performance.sav'
pickle.dump(model, open(file_name, 'wb'))